# Berlin relocation

My wife and I will be relocating to Berlin in two month's time and being unfamiliar with the city we would like recommendations on the best places to live within the city. For my final Capstone project, I will be using foursquare data to review the different options along with (if obtainable) rental price information for each region to help decide upon an optimum, affordable place to live.

___

### Source data

The data I intend to use to help me complete this task includes:
 - **Foursquare's** geolocation data of Berlin's amenities - this will help to quantify the number of amenities in each of the Berlin boroughs
 - **Immo scout's** rental prices for cities across Germany - this will provide rent per sqm in order to make comparisons between the Berlin boroughs
 - **Geospatial** coordinates for the boroughs of Berlin - this will enable me to see the locations on a folium map
 - My wife and my preferences for nearby **amenities** - to help indicate which boroughs contain the density of amenities we are seeking

___

In [1]:
#Import the relevant packages
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests

In [2]:
pd.set_option('display.max_columns', None)    #Show max columns for ease of reading outputs

In [3]:
#Read the Germany rent data file
DE_rent = pd.read_csv('immo_data.csv')

In [4]:
#Filter the Germany rent data file for just Berlin
berlin_rent = DE_rent[DE_rent['regio1']=='Berlin']

In [5]:
#Reformat the region's text and add "Berlin" to the string (to avoid confusion with other German cities)
berlin_rent['Neighbourhoods'] = berlin_rent['regio3'].str.replace('_',', ')
berlin_rent['Neighbourhoods'] = berlin_rent['Neighbourhoods'] + ", Berlin"

<ipython-input-5-f3077a7d4623>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  berlin_rent['Neighbourhoods'] = berlin_rent['regio3'].str.replace('_',', ')
<ipython-input-5-f3077a7d4623>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  berlin_rent['Neighbourhoods'] = berlin_rent['Neighbourhoods'] + ", Berlin"


In [6]:
berlin_neighbourhoods = berlin_rent['Neighbourhoods'].reset_index(drop=True).unique() #Assign the neighbourhoods column to a new series

Obtain the lat and long coordinates of the Berlin neighbourhoods using the geocoders library

In [7]:
rows=[] #Initialise the rows dataframe

geolocator = Nominatim(user_agent="berlin_locator")

for i in range(len(berlin_neighbourhoods)):
    location = geolocator.geocode(berlin_neighbourhoods[i])
    print(location)
    if location != None:
        rows.append([berlin_neighbourhoods[i],location.latitude,location.longitude])
    
berlin_coords = pd.DataFrame(rows,columns=['Neighbourhood','Latitude','Longitude'])

Staaken, Spandau, Berlin, 13591, Deutschland
Weißensee, Pankow, Berlin, 13086, Deutschland
Mitte, Berlin, Deutschland
U Platz der Luftbrücke, Mehringdamm, Kreuzberg, Friedrichshain-Kreuzberg, Berlin, 10965, Deutschland
Berlin, Willy-Brandt-Straße, Tiergarten, Mitte, Berlin, 10557, Deutschland
Grünau, Treptow-Köpenick, Berlin, 12527, Deutschland
Marzahn, Marzahn-Hellersdorf, Berlin, Deutschland
Köpenick, Treptow-Köpenick, Berlin, Deutschland
S Hohenschönhausen, Falkenberger Chaussee, Neu-Hohenschönhausen, Lichtenberg, Berlin, Deutschland
S+U Wuhletal, Altentreptower Straße, Hellersdorf, Marzahn-Hellersdorf, Berlin, 12619, Deutschland
S Schönhauser Allee, Schönhauser Allee, Gleimviertel, Prenzlauer Berg, Pankow, Berlin, 10439, Deutschland
Berliner Straße, Französisch Buchholz, Pankow, Berlin, 13127, Deutschland
Berlin, Tauentzienstraße, Charlottenburg, Charlottenburg-Wilmersdorf, Berlin, 10789, Deutschland
Mariendorf, Tempelhof-Schöneberg, Berlin, Deutschland
Neukölln, Berlin, Deutschlan

In [8]:
#Check the output dataframe
berlin_coords.head()

,Neighbourhood,Latitude,Longitude
0,"Staaken, Spandau, Berlin",52.532271,13.143367
1,"Weißensee, Weißensee, Berlin",52.553930,13.466022
2,"Mitte, Mitte, Berlin",52.517885,13.404060
3,"Kreuzberg, Kreuzberg, Berlin",52.486084,13.385951
4,"Tiergarten, Tiergarten, Berlin",52.520226,13.370487


In [9]:
# create the map of Berlin using the latitude and longitude values
map_berlin = folium.Map(location=[52.5, 13.4], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(berlin_coords['Latitude'], berlin_coords['Longitude'], berlin_coords['Neighbourhood']):
    label = folium.Popup(neighbourhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='darkblue',
        fill=True,
        fill_color='darkblue',
        fill_opacity=0.6,
        parse_html=False).add_to(map_berlin)  

map_berlin

___

**Foursquare**

In [10]:
# function that extracts the category of the venue, lifted from the labs
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
CLIENT_ID = 'ZND133FS1T01EY4UD52IT5SKKR3LHXOZWFVC4T5SIUGDVGRG' # Foursquare ID
CLIENT_SECRET = 'NKZAL25IPARP1VLESDB5PFPOPLAEMYQU235JYV01L2M5RNOZ' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Run the same scripts as the Toronto neighbourhoods lab...

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
#Run the above function on each Berlin neighborhood and create a new dataframe
berlin_venues = getNearbyVenues(names=berlin_coords['Neighbourhood'],
                                   latitudes=berlin_coords['Latitude'],
                                   longitudes=berlin_coords['Longitude']
                                  )

Staaken, Spandau, Berlin
Weißensee, Weißensee, Berlin
Mitte, Mitte, Berlin
Kreuzberg, Kreuzberg, Berlin
Tiergarten, Tiergarten, Berlin
Grünau, Köpenick, Berlin
Marzahn, Marzahn, Berlin
Köpenick, Köpenick, Berlin
Neu, Hohenschönhausen, Hohenschönhausen, Berlin
Hellersdorf, Hellersdorf, Berlin
Prenzlauer, Berg, Prenzlauer, Berg, Berlin
Französisch, Buchholz, Pankow, Berlin
Charlottenburg, Charlottenburg, Berlin
Mariendorf, Tempelhof, Berlin
Neukölln, Neukölln, Berlin
Schmargendorf, Wilmersdorf, Berlin
Wedding, Wedding, Berlin
Siemensstadt, Spandau, Berlin
Britz, Neukölln, Berlin
Buch, Pankow, Berlin
Friedrichshain, Friedrichshain, Berlin
Wilmersdorf, Wilmersdorf, Berlin
Reinickendorf, Reinickendorf, Berlin
Schmöckwitz, Köpenick, Berlin
Wittenau, Reinickendorf, Berlin
Spandau, Spandau, Berlin
Adlershof, Treptow, Berlin
Friedenau, Schöneberg, Berlin
Alt, Hohenschönhausen, Hohenschönhausen, Berlin
Schöneberg, Schöneberg, Berlin
Friedrichsfelde, Lichtenberg, Berlin
Karlshorst, Lichtenberg, B

In [26]:
# one hot encoding
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
berlin_onehot['Neighbourhood'] = berlin_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]

In [27]:
berlin_grouped = berlin_onehot.groupby('Neighbourhood').mean().reset_index()

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 6

# create columns according to number of top venues
columns = ['Neighbourhood']

for ind in np.arange(num_top_venues):
    columns.append('No.{} Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = berlin_grouped['Neighbourhood']

for ind in np.arange(berlin_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

Apply the k-means clustering algorithm from sklearn

In [30]:
# set number of clusters
kclusters = 4

berlin_grouped_clustering = berlin_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 2, 1, 2, 0, 1, 2], dtype=int32)

In [31]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

berlin_merged = berlin_coords

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
berlin_merged = berlin_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [32]:
# create the Berlin map with the clusters
map_clusters = folium.Map(location=[52.5, 13.4], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(berlin_merged['Latitude'], berlin_merged['Longitude'], berlin_merged['Neighbourhood'], berlin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1 (the purple dots)

In [33]:
cluster1 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 0, berlin_merged.columns[[4] + list(range(5, berlin_merged.shape[1]))]]
cluster1

,No.1 Most Common Venue,No.2 Most Common Venue,No.3 Most Common Venue,No.4 Most Common Venue,No.5 Most Common Venue,No.6 Most Common Venue
8,Supermarket,Drugstore,Gym / Fitness Center,Department Store,Seafood Restaurant,Electronics Store
9,Supermarket,Miscellaneous Shop,Light Rail Station,Trail,Plaza,Museum
13,Supermarket,German Restaurant,Drugstore,Fast Food Restaurant,Greek Restaurant,Chinese Restaurant
17,Supermarket,Furniture / Home Store,Fast Food Restaurant,Metro Station,Playground,Hardware Store
19,Supermarket,Drugstore,Bus Stop,Art Gallery,Bakery,Asian Restaurant
21,Supermarket,Hotel,Greek Restaurant,Café,Mexican Restaurant,Italian Restaurant
26,Drugstore,Supermarket,Italian Restaurant,Trattoria/Osteria,Shopping Mall,Steakhouse
28,Hotel,Drugstore,Vietnamese Restaurant,Supermarket,Memorial Site,Liquor Store
30,Drugstore,Supermarket,Metro Station,Zoo Exhibit,Market,Restaurant
38,Italian Restaurant,Supermarket,Drugstore,Light Rail Station,Gastropub,Fast Food Restaurant


#### Cluster 2 (the light blue dots)

In [34]:
cluster2 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 1, berlin_merged.columns[[4] + list(range(5, berlin_merged.shape[1]))]]
cluster2

,No.1 Most Common Venue,No.2 Most Common Venue,No.3 Most Common Venue,No.4 Most Common Venue,No.5 Most Common Venue,No.6 Most Common Venue
0,Eastern European Restaurant,Italian Restaurant,Supermarket,Restaurant,Bus Stop,Newsstand
22,Pool,Soccer Field,Bus Stop,German Restaurant,Metro Station,Supermarket
29,Bus Stop,Furniture / Home Store,Drugstore,Café,Gym / Fitness Center,Bakery
34,Supermarket,Sushi Restaurant,Pizza Place,Movie Theater,Taverna,Burger Joint
36,Italian Restaurant,Supermarket,Restaurant,Bus Stop,ATM,Newsstand
39,Soccer Field,Supermarket,Bus Stop,Doner Restaurant,Other Repair Shop,Museum
40,Supermarket,Pizza Place,German Restaurant,Bus Stop,Miscellaneous Shop,Music Venue
47,Bus Stop,Bakery,Supermarket,Park,Farmers Market,Fast Food Restaurant
60,Forest,Bus Stop,Home Service,Outdoor Sculpture,Museum,Music Store
67,Bank,Gym / Fitness Center,Park,Metro Station,Automotive Shop,Beer Store


#### Cluster 3 (the yellow dots)

In [79]:
cluster3 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 2, berlin_merged.columns[[4] + list(range(5, berlin_merged.shape[1]))]]
cluster3

,No.1 Most Common Venue,No.2 Most Common Venue,No.3 Most Common Venue,No.4 Most Common Venue,No.5 Most Common Venue,No.6 Most Common Venue
1,German Restaurant,Beach,Park,Indian Restaurant,Hotel,Bistro
2,Hotel,History Museum,Art Museum,Scenic Lookout,Café,Restaurant
3,Italian Restaurant,Café,Nightclub,Bar,Music Venue,Ice Cream Shop
4,Hotel,Sandwich Place,Bakery,Coffee Shop,Plaza,Scenic Lookout
5,Tram Station,Supermarket,Smoke Shop,Snack Place,Restaurant,Boat or Ferry
7,Clothing Store,Drugstore,Tram Station,Bank,Supermarket,Bakery
10,Café,Falafel Restaurant,Bar,Ice Cream Shop,Vietnamese Restaurant,German Restaurant
12,Hotel,Zoo Exhibit,Clothing Store,French Restaurant,Hotel Bar,Burger Joint
14,Bar,Café,Coffee Shop,Cocktail Bar,Italian Restaurant,Turkish Restaurant
15,Pool,Trattoria/Osteria,Pet Store,Fast Food Restaurant,Football Stadium,Moving Target


#### Cluster 4 (the red dots)

In [86]:
cluster4 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 3, berlin_merged.columns[[4] + list(range(5, berlin_merged.shape[1]))]]
cluster4

,No.1 Most Common Venue,No.2 Most Common Venue,No.3 Most Common Venue,No.4 Most Common Venue,No.5 Most Common Venue,No.6 Most Common Venue
6,Tram Station,Asian Restaurant,Plaza,German Restaurant,Windmill,Supermarket
11,Tram Station,Supermarket,Asian Restaurant,Multiplex,Museum,Music Store
41,Supermarket,Tram Station,Escape Room,Gym / Fitness Center,Italian Restaurant,Bowling Alley
45,Tram Station,Supermarket,Automotive Shop,Hotel,Bus Stop,German Restaurant
70,Tram Station,Ice Cream Shop,ATM,Museum,Music Store,Music Venue
73,Supermarket,Tram Station,Nature Preserve,Bus Stop,Auto Garage,Nightclub
74,Tram Station,Supermarket,Chinese Restaurant,Shopping Mall,Lake,Gas Station


Based on the above, cluster 1 appears to be the optimal boroughs

In [80]:
#Create a dataframe which lists the neighbourhoods in cluster 1
berlin_ideal = berlin_merged['Neighbourhood'][berlin_merged['Cluster Labels']==0].to_frame()

In [87]:
#Create a dataframe which averages the rents for Berlin and renames the columns of the dataframe to align with the dataframe above
avg_rent = berlin_rent[['Neighbourhoods','totalRent']].groupby('Neighbourhoods',as_index=False).mean().rename(columns={'totalRent':'avgRent','Neighbourhoods':'Neighbourhood'})

In [88]:
#Merge the ideal neighbourhoods list with the average rents and sort the results
berlin_ideal.merge(avg_rent).sort_values('avgRent',ascending=False)

,Neighbourhood,avgRent
11,"Dahlem, Zehlendorf, Berlin",2116.109091
16,"Wannsee, Zehlendorf, Berlin",1927.432973
5,"Wilmersdorf, Wilmersdorf, Berlin",1908.607033
14,"Rahnsdorf, Köpenick, Berlin",1525.583636
13,"Hermsdorf, Reinickendorf, Berlin",1254.286452
12,"Tempelhof, Tempelhof, Berlin",1129.882989
6,"Adlershof, Treptow, Berlin",1063.875556
10,"Lankwitz, Steglitz, Berlin",1056.022456
17,"Rudow, Neukölln, Berlin",1036.339032
4,"Buch, Pankow, Berlin",1007.536087
